# Performing Flux Sampling


In [ ]:
# read for all iMAT output models through a loop
# Define the folder path where your iMAT output MATLAB models are located
input_path = 'path/to/iMAT/output/folder'

# define the folder path where you wanna keep the output of flux sampling as csv files 
output_path = 'path/to/sapling/output/folder'

In [2]:
# note: this is running on a python virtual environment of version 3.9.6 set on terminal and activated via conda to be used as a kernel here 
# import the necessary packages 
import os
import pandas as pd
import cobra
from cobra.util.solver import solvers
from time import time
from cobra.io import load_matlab_model # for reading the matlab models
from cobra.sampling import OptGPSampler, ACHRSampler # for Flux sampling 

In [ ]:
# running flux sampling 
# for a number of models at the same time

for i, filename in enumerate(os.listdir(input_path)):
    if filename.endswith('.mat'): 
        print(f"Processing file {i + 1}/{len(os.listdir(input_path))}: {filename}")
        
        file_path = os.path.join(input_path, filename)
        
        try:
            # Load the iMAT model
            print(f"Loading model: {filename}")
            model = load_matlab_model(file_path)
            model.solver = 'gurobi'
            
            # Run FBA to check solution feasibility
            solution = model.optimize()
            if solution.status != 'optimal':
                print(f"Model {filename} is not optimal. Skipping...")
                continue
            
            # Perform flux sampling
            print(f"Sampling the model: {filename}")
            optgp = OptGPSampler(model, processes=4)
            s = optgp.sample(1000)
            
            # Convert sampling results to DataFrame
            df = pd.DataFrame(s)
            csv_filename = os.path.splitext(filename)[0] + '_sampling.csv'
            csv_file_path = os.path.join(output_path, csv_filename)
            df.to_csv(csv_file_path, index=False)
            print(f"Sampling results saved to: {csv_file_path}")
        
        except Exception as e:
            print(f"Error processing {filename}: {e}")
            continue

In [ ]:
# flux sampling for one model at a time
# Set Gurobi as the solver
cobra.Configuration().solver = "gurobi"

# Confirm the solver
print("Selected Solver:", cobra.Configuration().solver)

# Load your model
model = cobra.io.load_matlab_model('path/to/model.mat')

# Test optimization
solution = model.optimize()
print("Objective Value:", solution.objective_value)

In [ ]:
# run Flux Sampling
optgp = OptGPSampler(model, processes=4)
s = optgp.sample(1000)

In [ ]:
df = pd.DataFrame(s)
csv_filename = 'path/to/sampling/output/folder/name.csv'
csv_file_path = os.path.join(output_path, csv_filename)
df.to_csv(csv_file_path, index=False)